In [1]:
import re
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import GRU
from keras.optimizers import Adamax

import numpy as np
import random
import sys
import io
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
ghazals=pd.read_excel('urdu/faiz/ghazals.xlsx')
ghazals=ghazals.reset_index()
del ghazals['index']

In [3]:
#Because i want it to be purly urdu so i'll delete all the nazam's with english

ghazals_pruned=pd.DataFrame(columns=list(['title','text']))
for counter,i in enumerate(ghazals['text']):
    if(re.search('[a-zA-Z]', i))==None:
        df = pd.DataFrame([[ghazals['title'][counter], i]],columns=list(['title','text']))
        ghazals_pruned=ghazals_pruned.append(df, ignore_index=True)

In [4]:
#Finding the longest line in literature
longest=0
for i in ghazals_pruned['text']:
    #Finding the longest line in the literature
    for j in i.split('\n'):
        if longest<len(j):
            longest=len(j)
print("Longest line in the literature",longest)

Longest line in the literature 63


In [5]:
text=''
for i in ghazals_pruned['text']:
    #Creating the dataset
    text=text+i
print("Length of the literature:",len(text))

Length of the literature: 36245


In [6]:
new_text=''
for i in ghazals_pruned['text']:
    #Finding the longest line in the literature
    for j in i.split('\n'):
        if(len(j)<=2):
            continue
        j=j.strip()
        j=j.ljust(longest-1, ' ')
        j=j+'\n'
        new_text=new_text+j
ghazals_final=new_text

In [7]:
ghazals_final = ghazals_final.replace("'", "")

In [ ]:
nazams=pd.read_excel('urdu/faiz/nazams.xlsx')
nazams=nazams.reset_index()
del nazams['index']

In [9]:
#Because i want it to be purly urdu so i'll delete all the literature with english

nazams_pruned=pd.DataFrame(columns=list(['title','text']))
for counter,i in enumerate(nazams['text']):
    if(re.search('[a-zA-Z]', i))==None:
        df = pd.DataFrame([[nazams['title'][counter], i]],columns=list(['title','text']))
        nazams_pruned=nazams_pruned.append(df, ignore_index=True)

In [ ]:
nazams_pruned

In [11]:
#Finding the longest line in literature
# longest=0
# for i in nazams_pruned['text']:
#     #Finding the longest line in the literature
#     for j in i.split('\n'):
#         if longest<len(j):
#             longest=len(j)
# print("Longest line in the literature",longest)

In [12]:
text=''
for i in nazams_pruned['text']:
    #Creating the dataset
    text=text+i
print("Length of the literature:",len(text))

Length of the literature: 52424


In [13]:
new_text=''
for i in nazams_pruned['text']:
    #Finding the longest line in the literature
    for j in i.split('\n'):
        if(len(j)<=2):
            continue
        j=j.strip()
        j=j.ljust(longest-1, ' ')
        j=j+'\n'
        new_text=new_text+j
nazams_final=new_text

In [14]:
#some preprocessing
nazams_final = nazams_final.replace("'", "")
nazams_final = nazams_final.replace("(1)", "")
nazams_final = nazams_final.replace("(۲)", "")

In [17]:
#Finding unique characters
chars = sorted(list(set(nazams_final+ghazals_final)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 52


In [18]:
chars

['\n',
 ' ',
 '!',
 '،',
 'ؔ',
 '؟',
 'ء',
 'آ',
 'ؤ',
 'ئ',
 'ا',
 'ب',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'و',
 'ٓ',
 'ٔ',
 'ٰ',
 'ٹ',
 'پ',
 'چ',
 'ڈ',
 'ڑ',
 'ک',
 'گ',
 'ں',
 'ھ',
 'ہ',
 'ۂ',
 'ی',
 'ے',
 '۔']

In [19]:
len(ghazals_final+nazams_final)

169462

In [20]:
final_text=ghazals_final+nazams_final

In [21]:
maxlen = longest
step = longest
input_texts = []
target_texts = []
input_characters=set()
target_characters=set()

for i in range(0, len(final_text) - maxlen, step*2):
    input_text=final_text[i: i + maxlen]
    target_text=final_text[i + maxlen:i+maxlen + maxlen]
    target_text='\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


In [22]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [23]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1345
Number of unique input tokens: 51
Number of unique output tokens: 53
Max sequence length for inputs: 63
Max sequence length for outputs: 65


In [24]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [34]:
latent_dim=1024
batch_size=64
epochs=250

In [35]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))

encoder = LSTM(latent_dim, return_state=True)
# encoder = LSTM(latent_dim, return_state=True)(encoder)
# encoder = LSTM(latent_dim, return_state=True)(encoder)

encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [36]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, None, 51)     0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, None, 53)     0                                            
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, 1024), (None 4407296     input_8[0][0]                    
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, None, 1024), 4415488     input_9[0][0]                    
                                                                 lstm_9[0][1]                     
          

In [37]:
# Run training
model.compile(optimizer='adamax', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 1076 samples, validate on 269 samples
Epoch 1/250
1076/1076 [==============================] - 12s 11ms/step - loss: 3.0399 - val_loss: 2.0317
Epoch 2/250
1076/1076 [==============================] - 7s 6ms/step - loss: 1.9662 - val_loss: 1.9011
Epoch 3/250
1076/1076 [==============================] - 7s 6ms/step - loss: 1.9080 - val_loss: 1.8655
Epoch 4/250
1076/1076 [==============================] - 7s 6ms/step - loss: 1.8754 - val_loss: 1.8077
Epoch 5/250
1076/1076 [==============================] - 7s 7ms/step - loss: 1.8093 - val_loss: 1.7091
Epoch 6/250
1076/1076 [==============================] - 7s 7ms/step - loss: 1.7082 - val_loss: 1.7081
Epoch 7/250
1076/1076 [==============================] - 7s 7ms/step - loss: 1.6659 - val_loss: 1.7152
Epoch 8/250
1076/1076 [==============================] - 7s 7ms/step - loss: 1.6547 - val_loss: 1.6488
Epoch 9/250
1076/1076 [==============================] - 7s 6ms/step - loss: 1.6184 - val_loss: 1.6298
Epoch 10/250
1076/1076 [

KeyboardInterrupt: 

In [38]:
#Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [39]:
model.save_weights('seq2seq.hdf5')

In [40]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print(input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
شرح فراق مدح لب مشکبو کریں                                    

Decoded sentence: کر باد دلاں کو فری تے ترو گفر کر کر                           

-
یار آشنا نہیں کوئی ٹکرائیں کس سے جام                         

Decoded sentence:  س کو چلب سے کو ترے تر وور کر کریں                            

-
سینے پہ ہاتھ ہے نہ نظر کو تلاش بام                            

Decoded sentence:  س کو چلب سے کو ترے تر وور کر کریں                            

-
کب تک سنے گی رات کہاں تک سنائیں ہم                            

Decoded sentence: کر باد دلاں کو فرض تل گر گرو کریں                             

-
ہمدم حدیث کوئے ملامت سنائیو                                   

Decoded sentence: کر بات  بے آج کے در بر گرفگ رفی                               

-
آشفتہ سر ہیں محتسبو منہ نہ آئیو                              

Decoded sentence: کر باد دلاں کو فری تے تر وفگ عرو کریں                          

-
تر دامنی پہ شیخ ہماری نہ جائیو                              
دا
Decoded sentence:  کو چل دوں کے ترف تر گرو گر

In [ ]:
model.save_weights('seq2seq2.hdf5')